In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
from sklearn.pipeline import Pipeline
import pkg_resources
import glob
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np

/var/folders/dp/4gxlxdsj3s5gqzxbyb19tvl00000gn/T/ipykernel_5846/805670324.py:4: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [3]:
file_path = {
    "2013_Targa_Sixty_Six": "data/2013_Targa_Sixty_Six",
    "2014_Targa_Sixty_Six": "data/2014_Targa_Sixty_Six",
    "2013_Laguna_Seca": "data/2013_Laguna_Seca",
}

dflist = []

for i, fname in enumerate(sorted(glob.glob("data/2013_Targa_Sixty_Six" + "/*.csv"))):
    dflist.append(pd.read_csv(fname, sep=","))

testlist = [dflist[0][:5], dflist[1][:5]]

In [5]:
class ApplyThreshold(BaseEstimator, TransformerMixin):
    
    def __init__(self, threshold, seq_length=1, threshold_as_upper_bound=False):
        self.threshold = threshold
        self.seq_length = seq_length
        self.threshold_as_upper_bound = threshold_as_upper_bound

    def fit(self, X, y=None):
        return self

    def _threshold_condition(self, df):
        if self.threshold_as_upper_bound:
            return df["vxCG"] < self.threshold
        else:
            return df["vxCG"] > self.threshold

    def _process_group(self, group):
        if len(group) > self.seq_length:
            return group

    def transform(self, dflist):
        new_dflist = []
        for df in dflist:
            df_mask = self._threshold_condition(df)
            groups = df[df_mask].groupby((~df_mask).cumsum())
            new_data = [self._process_group(group) for _, group in groups if self._process_group(group) is not None]
            new_dflist.extend(new_data)

        return new_dflist

class _ConcatDataFrames(BaseEstimator,TransformerMixin):

    def fit(self, dflist, y = None):
        return self
    
    def transform(self, dflist):
        return pd.concat(dflist, keys = np.arange(0,len(dflist),1))

class _SeparateDataFrames(BaseEstimator,TransformerMixin):

    def fit(self, dflist, y = None):
        return self
    
    def transform(self, df):
        return [df.xs(i) for i in df.index.get_level_values(0).unique().to_list()]
    

class PL_StandardScaler(BaseEstimator,TransformerMixin):

    def __init__(self):
        self.scaler = StandardScaler().set_output(transform="pandas")
        return None

    def fit(self, dflist, y = None):
        self.scaler._reset()
        for df in dflist:
            self.scaler.partial_fit(df)
        return self
    
    def transform(self, dflist):
        scaled_df_list = []
        for df in dflist:
            scaled_df_list.append(self.scaler.transform(df))
        return scaled_df_list

class PL_MinMaxScaler(BaseEstimator,TransformerMixin):

    def __init__(self):
        self.scaler = MinMaxScaler().set_output(transform="pandas")
        return None

    def fit(self, dflist, y = None):
        self.scaler._reset()
        for df in dflist:
            self.scaler.partial_fit(df)
        return self
    
    def transform(self, dflist):
        scaled_df_list = []
        for df in dflist:
            scaled_df_list.append(self.scaler.transform(df))
        return scaled_df_list

In [12]:
testlist[0]

,sideSlip,vxCG,vyCG,time,longitude,latitude,engineSpeed,handwheelAngle,throttle,brake,axCG,ayCG,yawRate,chassisAccelFL,chassisAccelFR,chassisAccelRL,chassisAccelRR
0,0.0,0.0,-0.0,0.00,NaN,NaN,210,-22.3,0.5,0.0,0.04,-0.20,-0.21,9.56,9.61,9.01,9.14
1,0.0,0.0,-0.0,0.05,NaN,NaN,210,-22.2,0.5,0.0,-0.09,-0.03,-0.03,9.58,9.64,9.16,9.25
2,0.0,0.0,-0.0,0.10,NaN,NaN,210,-22.3,0.5,0.0,-0.03,-0.10,0.11,9.60,9.61,9.07,9.14
3,0.0,0.0,-0.0,0.15,NaN,NaN,210,-22.2,0.5,0.0,-0.02,-0.09,0.11,9.61,9.62,9.01,9.10
4,0.0,0.0,-0.0,0.20,NaN,NaN,210,-22.2,0.5,0.0,0.01,-0.14,-0.35,9.59,9.59,9.06,9.20


In [11]:
pipeline = Pipeline([('threshold', ApplyThreshold(threshold=10, seq_length=10))])
new_df_list = pipeline.fit_transform(testlist)

In [12]:
new_df_list

[]

In [13]:
pip1 = Pipeline([("test",concatDataFrames())])
concatdf = pip1.transform(testlist)

In [14]:
concatdf

sideSlip  vxCG  vyCG  time  longitude   latitude  engineSpeed  \
0 0       0.0  0.00 -0.00  0.00        NaN        NaN          210   
  1       0.0  0.00 -0.00  0.05        NaN        NaN          210   
  2       0.0  0.00 -0.00  0.10        NaN        NaN          210   
  3       0.0  0.00 -0.00  0.15        NaN        NaN          210   
  4       0.0  0.00 -0.00  0.20        NaN        NaN          210   
1 0       0.0 -0.01  0.00  0.00 -80.306057  26.920794         2178   
  1       0.0 -0.01 -0.01  0.05 -80.306057  26.920794         2088   
  2       0.0 -0.01 -0.02  0.10 -80.306057  26.920794         2112   
  3       0.0 -0.01  0.00  0.15 -80.306057  26.920794         2028   
  4       0.0 -0.01  0.01  0.20 -80.306057  26.920794         2148   

     handwheelAngle  throttle  brake  axCG  ayCG  yawRate  chassisAccelFL  \
0 0           -22.3       0.5    0.0  0.04 -0.20    -0.21            9.56   
  1           -22.2       0.5    0.0 -0.09 -0.03    -0.03            9.58   
  2           -22.3       0.5    0.0 -0.03 -0.10     0.11            9.60   
  3           -22.2       0.5    0.0 -0.02 -0.09     0.11            9.61   
  4           -22.2       0.5    0.0  0.01 -0.14    -0.35            9.59   
1 0           -13.6       2.2    0.0  0.22  0.57    -0.06           10.91   
  1           -13.7       2.2    0.0 -0.28  0.04     0.17            7.85   
  2           -13.7       2.2    0.0 -0.36 -0.94    -0.04            7.64   
  3           -13.6       2.3    0.0  0.09  0.24    -0.13            8.67   
  4           -13.5       2.3    0.0  0.08  0.24    -0.04           13.50   

     chassisAccelFR  chassisAccelRL  chassisAccelRR  
0 0            9.61            9.01            9.14  
  1            9.64            9.16            9.25  
  2            9.61            9.07            9.14  
  3            9.62            9.01            9.10  
  4            9.59            9.06            9.20  
1 0           10.04           10.06            9.76  
  1           10.85            8.88            8.60  
  2            9.06            8.24            9.06  
  3            8.82            9.02            9.30  
  4            7.96            8.90            9.13

In [15]:
pip2 = Pipeline([("test",separateDataFrames())])
dflist = pip2.transform(concatdf)

In [16]:
dflist[0]

,sideSlip,vxCG,vyCG,time,longitude,latitude,engineSpeed,handwheelAngle,throttle,brake,axCG,ayCG,yawRate,chassisAccelFL,chassisAccelFR,chassisAccelRL,chassisAccelRR
0,0.0,0.0,-0.0,0.00,NaN,NaN,210,-22.3,0.5,0.0,0.04,-0.20,-0.21,9.56,9.61,9.01,9.14
1,0.0,0.0,-0.0,0.05,NaN,NaN,210,-22.2,0.5,0.0,-0.09,-0.03,-0.03,9.58,9.64,9.16,9.25
2,0.0,0.0,-0.0,0.10,NaN,NaN,210,-22.3,0.5,0.0,-0.03,-0.10,0.11,9.60,9.61,9.07,9.14
3,0.0,0.0,-0.0,0.15,NaN,NaN,210,-22.2,0.5,0.0,-0.02,-0.09,0.11,9.61,9.62,9.01,9.10
4,0.0,0.0,-0.0,0.20,NaN,NaN,210,-22.2,0.5,0.0,0.01,-0.14,-0.35,9.59,9.59,9.06,9.20


In [6]:
testlist[0]

,sideSlip,vxCG,vyCG,time,longitude,latitude,engineSpeed,handwheelAngle,throttle,brake,axCG,ayCG,yawRate,chassisAccelFL,chassisAccelFR,chassisAccelRL,chassisAccelRR
0,0.0,0.0,-0.0,0.00,NaN,NaN,210,-22.3,0.5,0.0,0.04,-0.20,-0.21,9.56,9.61,9.01,9.14
1,0.0,0.0,-0.0,0.05,NaN,NaN,210,-22.2,0.5,0.0,-0.09,-0.03,-0.03,9.58,9.64,9.16,9.25
2,0.0,0.0,-0.0,0.10,NaN,NaN,210,-22.3,0.5,0.0,-0.03,-0.10,0.11,9.60,9.61,9.07,9.14
3,0.0,0.0,-0.0,0.15,NaN,NaN,210,-22.2,0.5,0.0,-0.02,-0.09,0.11,9.61,9.62,9.01,9.10
4,0.0,0.0,-0.0,0.20,NaN,NaN,210,-22.2,0.5,0.0,0.01,-0.14,-0.35,9.59,9.59,9.06,9.20


In [22]:
pip3 = Pipeline([("test",PL_StandardScaler())])
scaled_df = pip3.fit_transform(testlist)

/Users/jensbrandt/Desktop/Venvs/phd_repo/lib/python3.11/site-packages/sklearn/utils/extmath.py:1050: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/jensbrandt/Desktop/Venvs/phd_repo/lib/python3.11/site-packages/sklearn/utils/extmath.py:1055: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/Users/jensbrandt/Desktop/Venvs/phd_repo/lib/python3.11/site-packages/sklearn/utils/extmath.py:1075: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [32]:

pd.concat([scaled_df[0], scaled_df[1]]).describe()

,sideSlip,vxCG,vyCG,time,longitude,latitude,engineSpeed,handwheelAngle,throttle,brake,axCG,ayCG,yawRate,chassisAccelFL,chassisAccelFR,chassisAccelRL,chassisAccelRR
count,10.0,10.000000,1.000000e+01,1.000000e+01,0.0,0.0,1.000000e+01,1.000000e+01,1.000000e+01,10.0,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01,1.000000e+01
mean,0.0,0.000000,-8.881784e-17,-2.220446e-17,NaN,NaN,-1.110223e-16,-6.661338e-17,1.332268e-16,0.0,7.771561e-17,4.440892e-17,2.359224e-17,4.440892e-17,-7.993606e-16,-4.607426e-16,-3.330669e-15
std,0.0,1.054093,1.054093e+00,1.054093e+00,NaN,NaN,1.054093e+00,1.054093e+00,1.054093e+00,0.0,1.054093e+00,1.054093e+00,1.054093e+00,1.054093e+00,1.054093e+00,1.054093e+00,1.054093e+00
min,0.0,-1.000000,-2.405351e+00,-1.414214e+00,NaN,NaN,-9.992659e-01,-1.013812e+00,-9.992082e-01,0.0,-1.982358e+00,-2.413834e+00,-2.032642e+00,-1.280313e+00,-2.107050e+00,-1.918328e+00,-2.120407e+00
25%,0.0,-1.000000,2.672612e-01,-7.071068e-01,NaN,NaN,-9.992659e-01,-9.906126e-01,-9.992082e-01,0.0,-2.493150e-01,-2.389669e-01,-4.393994e-01,-4.829027e-01,-3.985375e-01,-2.718230e-01,-2.258532e-01
50%,0.0,0.000000,2.672612e-01,0.000000e+00,NaN,NaN,-4.352863e-02,-4.639872e-03,-2.297030e-02,0.0,1.763447e-01,-5.101540e-02,4.695872e-02,-4.201922e-02,1.802083e-01,-6.226781e-02,-1.045271e-01
75%,0.0,1.000000,2.672612e-01,7.071068e-01,NaN,NaN,9.942191e-01,9.987324e-01,9.532676e-01,0.0,6.324088e-01,6.202398e-01,8.184234e-01,-2.769449e-02,2.148637e-01,6.346527e-02,2.594512e-01
max,0.0,1.000000,1.603567e+00,1.414214e+00,NaN,NaN,1.069921e+00,1.027732e+00,1.068119e+00,0.0,1.544537e+00,1.640548e+00,1.455720e+00,2.450485e+00,1.899118e+00,2.440419e+00,2.210002e+00
